In [2]:
import fasttext

In [3]:
model_1k = fasttext.load_model('/workspace/search_with_machine_learning_course/week3/query_classifier_1k.bin')

In [6]:
model_1k.predict('ipad 5', k=3)

(('__label__pcmcat218000050003',
  '__label__pcmcat209000050007',
  '__label__pcmcat218000050000'),
 array([0.44220558, 0.43358949, 0.03764988]))

In [8]:
model_1k.predict('pyaaz', k=3)

(('__label__cat02015',
  '__label__pcmcat171900050028',
  '__label__pcmcat200900050008'),
 array([0.13736513, 0.05753118, 0.05580526]))

In [9]:
model_1k.predict('ipad', k=3)

(('__label__pcmcat209000050007',
  '__label__pcmcat209000050008',
  '__label__pcmcat218000050003'),
 array([0.69040233, 0.12954742, 0.02360768]))

# 10k min_queries model

In [4]:
model_10k = fasttext.load_model('/workspace/search_with_machine_learning_course/week3/query_classifier_10k.bin')

In [12]:
model_10k.predict('ipad 5', k=3)

(('__label__pcmcat217900050000',
  '__label__pcmcat209000050007',
  '__label__abcat0201011'),
 array([0.60524142, 0.19092394, 0.09940686]))

In [13]:
model_10k.predict('ipad', k=3)

(('__label__pcmcat209000050007',
  '__label__pcmcat209000050008',
  '__label__pcmcat217900050000'),
 array([0.63250387, 0.16552548, 0.09696095]))

In [14]:
model_10k.predict('pyaaz', k=3)

(('__label__cat02015', '__label__cat00000', '__label__cat02001'),
 array([0.15642525, 0.09264985, 0.09007225]))

In [7]:
import pandas as pd
import xml.etree.ElementTree as ET



In [9]:
categories_file_name = r'/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml'
root_category_id = 'cat00000'

In [46]:
tree = ET.parse(categories_file_name)
root = tree.getroot()

# Parse the category XML file to map each category id to its parent category id in a dataframe.
categories = []
parents = []
crumbs = []
for child in root:
    id = child.find('id').text
    cat_path = child.find('path')
    cat_path_ids = [cat.find('id').text for cat in cat_path]
    leaf_id = cat_path_ids[-1]
    if leaf_id != root_category_id:
        categories.append(leaf_id)
        parents.append(cat_path_ids[-2])
    paths = []
    for cat in cat_path:
        paths.append(cat.find('name').text)
    crumbs.append(" > ".join(paths))

categories.append(root_category_id)
parents.append(root_category_id)
crumbs.append('Best Buy')

parents_df = pd.DataFrame(list(zip(categories, parents, crumbs)), columns =['category', 'parent', 'crumbs'])

In [47]:
parents_df

,category,parent,crumbs
0,abcat0010000,cat00000,Best Buy > Gift Center
1,abcat0011000,abcat0010000,Best Buy > Gift Center > Her
2,abcat0011001,abcat0011000,Best Buy > Gift Center > Her > Leisure Gifts
3,abcat0011002,abcat0011000,Best Buy > Gift Center > Her > Kitchen Essentials
4,abcat0011003,abcat0011000,Best Buy > Gift Center > Her > Electronics
...,...,...,...
4635,pcmcat97200050015,cat15063,Best Buy > Name Brands > Sony > |Computer & Ph...
4636,pcmcat99000050001,pcmcat50000050006,Best Buy > Name Brands > Sony > Sony DSLR Camera
4637,pcmcat99000050002,pcmcat99000050001,Best Buy > Name Brands > Dynex > Networking
4638,pcmcat99300050000,cat15063,Best Buy > Name Brands > Dynex > Networking > ...


In [53]:
cats = parents_df.set_index('category')

In [54]:
cats

,parent,crumbs
category,,
abcat0010000,cat00000,Best Buy > Gift Center
abcat0011000,abcat0010000,Best Buy > Gift Center > Her
abcat0011001,abcat0011000,Best Buy > Gift Center > Her > Leisure Gifts
abcat0011002,abcat0011000,Best Buy > Gift Center > Her > Kitchen Essentials
abcat0011003,abcat0011000,Best Buy > Gift Center > Her > Electronics
...,...,...
pcmcat97200050015,cat15063,Best Buy > Name Brands > Sony > |Computer & Ph...
pcmcat99000050001,pcmcat50000050006,Best Buy > Name Brands > Sony > Sony DSLR Camera
pcmcat99000050002,pcmcat99000050001,Best Buy > Name Brands > Dynex > Networking


In [19]:
model_1k.predict('ipad 5', k=3)

(('__label__pcmcat218000050003',
  '__label__pcmcat209000050007',
  '__label__pcmcat218000050000'),
 array([0.44220558, 0.43358949, 0.03764988]))

In [61]:
def predict_cat(keyword, model=model_1k, k = 3):
    (preds, probs) = model.predict(keyword, k=k)
    pred_cats = [cats.loc[pred.replace('__label__', '')] for pred in preds]
    crumbs = [pc['crumbs'] for pc in pred_cats]
    return list(zip(probs, crumbs))

In [70]:
from pprint import pp

In [73]:
pp(predict_cat('ipad 5', model=model_1k, k=3))

[(0.4422055780887604,
  'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories > iPad '
  'Cables, Chargers & Adapters'),
 (0.43358948826789856, 'Best Buy > Computers & Tablets > Tablets & iPad'),
 (0.03764987736940384,
  'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories')]


In [74]:
print('ipad 5', predict_cat('ipad 5', model=model_1k, k=3))
print("\n")
print('ipad 5', predict_cat('ipad 5', model=model_10k, k=3))

ipad 5 [(0.4422055780887604, 'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories > iPad Cables, Chargers & Adapters'), (0.43358948826789856, 'Best Buy > Computers & Tablets > Tablets & iPad'), (0.03764987736940384, 'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories')]


ipad 5 [(0.6052414178848267, 'Best Buy > Computers & Tablets > test'), (0.19092394411563873, 'Best Buy > Computers & Tablets > Tablets & iPad'), (0.09940686076879501, 'Best Buy > Appliances > Fitness > Biking > All iPod & MP3 Players')]


In [34]:
'ipad 5 cover', predict_cat('ipad 5 cover')

('ipad 5 cover',
 [(0.8917661309242249, 'iPad Accessories > iPad Cases, Covers & Sleeves'),
  (0.030240604653954506, 'Mobile Phone Cases & Clips > Fitted'),
  (0.01598954200744629, 'Tablets & iPad > iPad Accessories')])

In [39]:
'iphone 5', predict_cat('iphone 5', model=model_1k, k=3)

('iphone 5',
 [(0.3063027858734131, 'Mobile Phones > All Mobile Phones with Plans'),
  (0.20283962786197662, 'Surface & Screen Protection > Screen Protectors'),
  (0.16612371802330017, 'iPhone Accessories > Cases')])

In [44]:
'iphone 5', predict_cat('iphone 5', model=model_10k, k=3)

('iphone 5',
 [(0.4193388521671295, 'Mobile Phones > All Mobile Phones with Plans'),
  (0.33154571056365967, 'Best Buy > Appliances'),
  (0.19655965268611908, 'Biking > All iPod & MP3 Players')])

In [45]:
'ipad 5 cover', predict_cat('ipad 5 cover', model=model_10k, k=3)

('ipad 5 cover',
 [(0.9808470606803894, 'Tablets & iPad > Tablet Accessories'),
  (0.010697782970964909, 'Tablets & iPad > iPad Accessories'),
  (0.006104671396315098, 'Tablets & iPad > Tablets')])

In [75]:
'iPad 5 cOvEr', predict_cat('iPad 5 cOvEr', model=model_10k, k=3)

('iPad 5 cOvEr',
 [(0.08316085487604141, 'Best Buy > Audio & MP3 > Car Audio'),
  (0.07396192103624344,
   'Best Buy > Computers & Tablets > Computer Cards & Components'),
  (0.06074994057416916, 'Best Buy > Computers & Tablets > Hard Drives')])

In [76]:
query = 'iPad 5 cOvEr'

In [78]:
import re

In [79]:
query = query.strip().lower()
query = re.sub(r"[^a-zA-Z0-9\s]", "", query)
query = re.sub(r"\s{2,}", " ", query)

In [80]:
query

'ipad 5 cover'

In [81]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")


In [82]:
stemmer.stem(query)

'ipad 5 cov'

In [83]:
def normalize_query(query):
    query = query.strip().lower()
    query = re.sub(r"[^a-zA-Z0-9\s]", "", query)
    query = re.sub(r"\s{2,}", " ", query)
    query = stemmer.stem(query)
    return query

In [84]:
keyword = 'ipad 5 cover'
keyword, predict_cat(normalize_query(keyword), model=model_10k, k=3)

('ipad 5 cover',
 [(0.8333727717399597, 'Best Buy > Computers & Tablets > test'),
  (0.06417825073003769, 'Best Buy > Computers & Tablets'),
  (0.021915797144174576,
   'Best Buy > TV & Home Theater > Home Theater Systems > Smart  Home Theater Systems')])

In [85]:
keyword = 'ipad 5 cover'
keyword, predict_cat(normalize_query(keyword), model=model_1k, k=3)

('ipad 5 cover',
 [(0.7835549116134644,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories'),
  (0.07657229155302048,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories > iPad Cables, Chargers & Adapters'),
  (0.03840577229857445,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories > iPad Cases, Covers & Sleeves')])

In [89]:
model_1k.predict(keyword, k=3)

(('__label__pcmcat218000050000',
  '__label__pcmcat171900050029',
  '__label__pcmcat217900050000'),
 array([0.89176613, 0.0302406 , 0.01598954]))

In [90]:
keyword = 'ipad 5'
keyword, predict_cat(normalize_query(keyword), model=model_1k, k=3)

('ipad 5',
 [(0.4422055780887604,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories > iPad Cables, Chargers & Adapters'),
  (0.43358948826789856, 'Best Buy > Computers & Tablets > Tablets & iPad'),
  (0.03764987736940384,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad Accessories')])

In [91]:
keyword = 'iphone'
keyword, predict_cat(normalize_query(keyword), model=model_1k, k=3)

('iphone',
 [(0.501879870891571,
   'Best Buy > Cameras & Camcorders > Camcorders > Shoot & Share Camcorders'),
  (0.07559408992528915,
   'Best Buy > Audio & MP3 > Marine Audio > Marine Speakers'),
  (0.06835659593343735,
   'Best Buy > Mobile Phones > Mobile Phone Accessories > Surface & Screen Protection > Surface Protectors')])

In [92]:
keyword = 'samsung galaxy'
keyword, predict_cat(normalize_query(keyword), model=model_1k, k=3)

('samsung galaxy',
 [(0.4715096354484558,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad'),
  (0.1247047707438469,
   'Best Buy > Best Buy For Business > Small Business & Home Office Essentials > Mobile Office > Laptop Accessories > Wireless Remote Presenters'),
  (0.10422579199075699,
   'Best Buy > Cameras & Camcorders > Camcorders > Shoot & Share Camcorders')])

In [93]:
keyword = 'samsung galaxy'
keyword, predict_cat(normalize_query(keyword), model=model_10k, k=3)

('samsung galaxy',
 [(0.5117916464805603,
   'Best Buy > Computers & Tablets > Tablets & iPad > iPad'),
  (0.165622279047966,
   'Best Buy > Cameras & Camcorders > Camcorders > Shoot & Share Camcorders'),
  (0.11534540355205536,
   'Best Buy > TV & Home Theater > Home Theater Systems > Smart  Home Theater Systems')])